In [1]:
from prepare.receptors import *
from prepare.constants import SEQRES, MINIMUM_FRAGMENT_SIZE
from prepare.dynamics import setup_fah_run
from pathlib import Path
from openeye import oechem, oespruce
import pandas as pd
import tempfile

In [2]:
config = PreparationConfig(input=Path('Mpro-P0033.pdb'), output=Path('Mpro-P0033'), is_dimer=True, retain_water=False)
output_filenames = create_output_filenames(config)

In [3]:
ligand = read_pdb_file(str(output_filenames.ligand_pdb))

In [4]:
# path = Path('Mpro-P0033.pdb')
# lines = config.input.open().readlines()

In [5]:
# errfs = oechem.oeosstream() # create a stream that writes internally to a stream
# oechem.OEThrow.SetOutputStream(errfs)
# oechem.OEThrow.Clear()
# oechem.OEThrow.SetLevel(oechem.OEErrorLevel_Verbose)

Remove extraneous bits from PDB

In [6]:
# lines = remove_from_lines(lines, 'UNK')
# lines = remove_from_lines(lines, 'HOH')
# lines = remove_from_lines(lines, 'LINK')
# if not is_in_lines(lines, 'SEQRES'):
#     lines = add_prefix(lines, SEQRES)
# mol = lines_to_mol_graph(lines)

Spruce to create docking system. This removes the second ligand.  

In [7]:
# options = set_options()
# metadata = oespruce.OEStructureMetadata()

# design_units = make_design_units(mol, metadata, options)
# docking_sys = make_docking_system(design_units[0])
# write_docking_system(docking_sys, output_filenames, is_thiolate=False)
# print(errfs)

Add SD metadata. 

In [8]:
md = pd.read_csv('/Users/robertarbon/Documents/Research/covid-moonshot/Mpro/metadata.csv')
md.drop(md.filter(regex='.*Unnamed.*', axis=1).columns, axis=1, inplace=True)

In [9]:
metadata = dict(md.loc[md.RealCrystalName == str(config.input.stem), :].iloc[0, :])
metadata = {k: v for k, v in metadata.items() if v is not np.nan}

In [10]:
# ligand = docking_sys.ligand
ligand.SetTitle(metadata['alternate_name'])
for k, v in metadata.items():
    oechem.OESetSDData(ligand, k, v)

In [11]:
output_filenames

OutputPaths(receptor_gzipped=PosixPath('Mpro-P0033/Mpro-P0033-receptor.oeb.gz'), receptor_thiolate_gzipped=PosixPath('Mpro-P0033/Mpro-P0033-receptor-thiolate.oeb.gz'), protein_pdb=PosixPath('Mpro-P0033/Mpro-P0033-protein.pdb'), protein_thiolate_pdb=PosixPath('Mpro-P0033/Mpro-P0033-protein-thiolate.pdb'), ligand_pdb=PosixPath('Mpro-P0033/Mpro-P0033-ligand.pdb'), ligand_sdf=PosixPath('Mpro-P0033/Mpro-P0033-ligand.sdf'), ligand_mol2=PosixPath('Mpro-P0033/Mpro-P0033-ligand.mol2'))

In [12]:
with tempfile.TemporaryDirectory() as tmpdir:
    cache = os.path.join(tmpdir, 'cache.json')
    setup_fah_run('Mpro-P0033', str(output_filenames.protein_pdb), oemol=ligand, cache=cache, biounit='dimer')

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): MIC-UNK-91acba05-6
Problematic atoms are:
Atom atomic num: 7, name:  N  , idx: 6, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name:  C19, idx: 5, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name:  C  , idx: 7, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name:  C1 , idx: 11, aromatic: False, chiral: False
Atom atomic num: 6, name:  C3 , idx: 9, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name:  C14, idx: 8, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name:  C2 , idx: 10, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name:  C4 , idx: 12, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name:  H3

[<Residue 0 (MOL) of chain 0>]
Reading protein from Mpro-P0033/Mpro-P0033-protein.pdb...
Adding solvent...


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): MOL
Problematic atoms are:
Atom atomic num: 7, name: , idx: 6, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 5, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 7, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 11, aromatic: False, chiral: False



Creating OpenMM system...
Creating virtual bond between atoms 0 and 4685
Creating virtual bond between atoms 0 and 9353
Initial potential energy is -158253.664 kcal/mol
Minimizing...


Output()

KeyboardInterrupt: 